In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    groq_api_key="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",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
response=llm.invoke("The first person to land on the moon was and who was second?")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969.

The second person to walk on the moon was Edwin "Buzz" Aldrin. He joined Neil Armstrong on the moon's surface shortly after Armstrong's historic first steps.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/content/en/career-programs/university")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


































Amazon University Talent Acquisition (AUTA)
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsDiversity, equity, and inclusionInterview tipsLeadership principlesWorking at AmazonFAQ× Amazon University TalentInfluence the Amazon of tomorrowAmazon internships and full-time roles for graduates give you the opportunity to solve problems, innovate on behalf of our customers, and shape our business.Find open university rolesAboutDesign your careerAt Amazon, your career will grow in new and exciting directions—designed by you. You’ll have the opportunity to be part of innovative projects, lead initiatives, and take ownership and responsibility.Here, you’ll work with groundbreaking technology and learn from the best all over the world, united with the common goal to delight our customers.Our internship roles for students, and our full-time roles for grad

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE
    {page_data}
    ###INSTRUCTION:
    you scraped text from the carrer's page of a website.
    your job is to extract the job postings and return them in json format
    containing following keys:`role`,`experience`,`skills` and `description`.
    Only return the valid JSON.
    ###VALID JSON (NO PREAMBLE)"""
)
chain_extract=prompt_extract | llm
res=chain_extract.invoke(input={'page_data'})
print(res.content)

```json
[
  {
    "role": "Software Engineer",
    "experience": "3-5 years",
    "skills": ["Java", "Spring Boot", "MySQL"],
    "description": "We are looking for a skilled Software Engineer to join our team. The ideal candidate will have experience in Java, Spring Boot, and MySQL. Responsibilities include designing, developing, and testing software applications."
  },
  {
    "role": "Data Scientist",
    "experience": "5-7 years",
    "skills": ["Python", "Machine Learning", "Data Visualization"],
    "description": "We are seeking a highly skilled Data Scientist to join our team. The ideal candidate will have experience in Python, Machine Learning, and Data Visualization. Responsibilities include analyzing complex data sets, developing predictive models, and creating data visualizations."
  },
  {
    "role": "DevOps Engineer",
    "experience": "2-4 years",
    "skills": ["AWS", "Docker", "Kubernetes"],
    "description": "We are looking for a skilled DevOps Engineer to join our 

In [4]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res=json_res[0]

In [5]:
json_res

{'role': 'Software Engineer',
 'experience': '3-5 years',
 'skills': ['Java', 'Spring Boot', 'MySQL'],
 'description': 'We are looking for a skilled Software Engineer to join our team. The ideal candidate will have experience in Java, Spring Boot, and MySQL. Responsibilities include designing, developing, and testing software applications.'}

In [6]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import chromadb
import uuid

client= chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="porfolio")
if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links":row["Links"]},
                      ids=[str(uuid.uuid4())])
                            

In [9]:
links=collection.query(query_texts=['Java', 'Python', 'AWS'],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [10]:
job=json_res
job['skills']


['Java', 'Spring Boot', 'MySQL']

In [11]:
job

{'role': 'Software Engineer',
 'experience': '3-5 years',
 'skills': ['Java', 'Spring Boot', 'MySQL'],
 'description': 'We are looking for a skilled Software Engineer to join our team. The ideal candidate will have experience in Java, Spring Boot, and MySQL. Responsibilities include designing, developing, and testing software applications.'}

In [12]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Shubham Baghel, a skilled AI and software developer from NIT Kurukshetra with a strong focus on leveraging AI to solve practical problems. 
    You have built tools like a Cold Email Generator using ChromaDB, LLMs, and Groq Cloud, as well as an AI-powered Resume Tailoring Tool 
    that optimizes resumes using Llama 3.1 and semantic matching. 
    Your goal is to write a cold email addressing the job requirements mentioned above, showcasing how your expertise in AI-powered solutions, 
    semantic search, and tailored automation can fulfill the client's needs. 
    Highlight your skills in Python, Streamlit, and LangChain, and demonstrate your ability to design and deploy cutting-edge solutions. 
    Include links to your most relevant projects: {link_list}.
    Do not provide a preamble.
    
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Revolutionizing Software Development with AI-Powered Solutions

Dear Hiring Manager,

I'm excited to apply for the Software Engineer position at your esteemed organization. With a strong background in AI and software development from NIT Kurukshetra, I'm confident that my expertise in designing and deploying cutting-edge solutions can fulfill your needs.

As a skilled developer with experience in building AI-powered tools, I've developed a unique ability to leverage AI to solve practical problems. My proficiency in Python, Streamlit, and LangChain enables me to create tailored automation solutions that streamline processes and enhance efficiency.

I understand that your organization is looking for a candidate with experience in Java, Spring Boot, and MySQL. Although my primary expertise lies in Python, I'm eager to adapt my skills to your technology stack. My experience in building AI-powered solutions has taught me the importance of semantic search and matching, which I belie